
**[PT]** Português

---

**[EN]** English


# Correção, inferência e normalização do campo "Faculdade"


---


# Correction, inference and normalization of the field "Faculdade"



## Setup

In [1]:
from timelink.api.database import TimelinkDatabase
from timelink import version
from ucalumni.config import default_db_url

print(f"Timelink API version: {version}")
print(f"Creating TimelinkDatabase instance from {default_db_url}")
db = TimelinkDatabase(db_url=default_db_url)


Timelink API version: 1.1.14
Creating TimelinkDatabase instance from sqlite:///../database/sqlite3/fauc3.db?check_same_thread=False


## Sobre o registo da Faculdade e das Matrículas nas fontes originais

Ver _Percurso académico na Universidade de Coimbra, nos séculos XVI a XX
(orientações para pesquisa)_
Ana Maria Leitão Bandeira, Arquivo da Universidade de Coimbra, https://www.uc.pt/auc/orientacoes/UC_GuiaPercursoAcademico.pdf

### Até 1772

> Até à Reforma Pombalina da Universidade em 1772 o registo de matrícula apresenta geralmente a seguinte informação:
Nome do aluno, naturalidade e filiação, sendo neste último caso apresentado apenas o nome do
pai, seguido da informação sobre a data de matrícula e assinatura do aluno. No caso de alunos de ordens
religiosas não é apresentado o dado de filiação.


### Depois de 1772

> Depois de 1772 são acrescentados os registos de matrícula das Faculdades de Matemática e
Filosofia e deixa de existir a cadeira de Instituta. Os registos passam a ser apresentados por ordem
alfabética e por ordem de ano de curso.

> O registo de matrícula apresenta ainda informação sobre a classe de ordinário ou de obrigado de
cada aluno. Quanto aos alunos voluntários existem volumes próprios para o registo da matrícula nas
Faculdades de Filosofia e de Matemática. Pode, eventualmente, surgir a informação, sobre a mudança de
uma classe para outra, designada por trânsito do aluno, por exemplo de ordinário para obrigado, ou de
voluntário para ordinário.

### Depois de 1793

> A partir de 1793 passam a ser utilizados formulários impressos,
com espaços em branco para neles se registar o nome do aluno, naturalidade e filiação. Na matrícula que
ocorre no primeiro ano do curso ficou ainda regista a informação sobre as habilitações do aluno, em
estudos preparatórios para o ingresso na Universidade (por exemplo: Latim, Retórica, Filosofia,
Geometria).

### Depois de 1911

> Após a reforma de 1911, de acordo com o Decreto com força de lei de 19 de Abril de 1911,
no seu cap. IX, a matrícula é o acto pelo qual o aluno dá entrada na Universidade; por seu lado,
a inscrição permite ao aluno, depois de matriculado, a frequência das diversas cadeiras e cursos.
A partir desta data, os livros de matrículas passam a estar ordenados por cada Faculdade e não é dada
indicação do ano de curso, uma vez que a matrícula diz respeito ao ingresso na Universidade e
respectiva Faculdade. Até 1937 não indicam o ano de curso, ou mesmo o próprio curso a
frequentar dentro de cada Faculdade. Estes dados só podem ser colhidos nos livros de
Inscrições.



## Notas de processamento

As notas de processamento são criadas pelo algoritmo de extração do valor da Faculdade.

Contém informação sobre inferências, correções automáticas e erros que necessitam de revisão.

Em termos narrativos, o algoritmo verifica primeiro se o campo "Faculdade" tem ou não um valor; depois procede à recolha de todas as referências a faculdades no resto do registo; se ao remover os pré-requisitos da recolha algo permanece, será o valor escolhido como faculdade; se nada permanece, então o valor original é usado; alguns outros casos especiais são testados. 


--- 

## Processing notes

The processing notes are created to register inferences, corrections and errors that need revision.

In narrative terms, the algorithm checks first if the field “Faculdade” has a value or not; it then proceeds to collect all references to “faculdades” in the rest of the record; if by removing the pre-requisites from the collection something remains, it will the chosen as the corrected or inferred “faculdade”; if nothing remains then the original value is used; some other edge cases are tested. 




In [2]:
from timelink.pandas import entities_with_attribute

notas_proc = entities_with_attribute(
                    entity_type='person',
                    the_type='nota-processamento',
                    show_elements=['id','name'],
                    more_attributes=['faculdade','faculdade-original','uc-entrada', 'uc-saida','url'],
                    sql_echo=False,
                    db=db)

notas_proc['id'] = notas_proc.index.values
notas_proc.dropna(how='all', axis=1, inplace=True)
notas_proc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24687 entries, 140351 to 196210
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id_1                      24687 non-null  object
 1   name                      24687 non-null  object
 2   nota-processamento.type   24687 non-null  object
 3   nota-processamento        24687 non-null  object
 4   nota-processamento.date   24687 non-null  object
 5   nota-processamento.line   24687 non-null  int64 
 6   nota-processamento.level  24687 non-null  int64 
 7   nota-processamento.obs    24687 non-null  object
 8   faculdade                 24012 non-null  object
 9   faculdade.date            24012 non-null  object
 10  faculdade.obs             24012 non-null  object
 11  faculdade-original        11899 non-null  object
 12  faculdade-original.date   11899 non-null  object
 13  faculdade-original.obs    11899 non-null  object
 14  uc-entrada           

In [3]:
df = notas_proc
df['id'] = df.index.values

col = 'nota-processamento' # sub total by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc-entrada':'min',
                                                  'uc-saida':'max'})
df_totals.loc['Total'] = df_totals['id'].sum()

df_totals.sort_values('id',ascending= False).head(30)

,id,uc-entrada,uc-saida
nota-processamento,,,
Total,19963,19963,19963
Aviso: faculdade inferida,11363,0000-00-00,1913-07-11
Aviso: faculdade corrigida,6971,0000-00-00,1916-07-15
Erro: não foi possível determinar a faculdade,675,1537-00-00,1912-08-02
"Faculdade corrigida, combinação incomum",564,0000-00-00,1916-07-15
Erro: Valor não corresponde a faculdade neste período,390,0000-00-00,1913-08-13



### Faculdade inferida

Casos em que o campo da faculdade não foi preenchido e o algoritmo sugere um valor.

---

### "Faculdade" Inferred


Cases where the value for faculty was missing and the algoritm suggested a value.



In [4]:
fac_inf = notas_proc[notas_proc['nota-processamento']=='Aviso: faculdade inferida'].copy()
fac_inf.dropna(how='all', axis=1, inplace=True)
fac_inf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11767 entries, 140351 to 196148
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id_1                      11767 non-null  object
 1   name                      11767 non-null  object
 2   nota-processamento.type   11767 non-null  object
 3   nota-processamento        11767 non-null  object
 4   nota-processamento.date   11767 non-null  object
 5   nota-processamento.line   11767 non-null  int64 
 6   nota-processamento.level  11767 non-null  int64 
 7   nota-processamento.obs    11767 non-null  object
 8   faculdade                 11767 non-null  object
 9   faculdade.date            11767 non-null  object
 10  faculdade.obs             11767 non-null  object
 11  uc-entrada                11767 non-null  object
 12  uc-entrada.date           11767 non-null  object
 13  uc-entrada.obs            11767 non-null  object
 14  uc-saida             

In [5]:
df = fac_inf
col = 'faculdade' # subotal by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc-entrada':'min',
                                                  'uc-saida':'max'})
print("List of inferred 'faculdade'")
df_totals.sort_values('id',ascending= False).head(30)

List of inferred 'faculdade'


,id,uc-entrada,uc-saida
faculdade,,,
Cursos jurídicos (Cânones ou Leis),6993,0000-00-00,1829-03-28
Artes,3010,0000-00-00,1780-07-18
Cânones,764,0000-00-00,1827-06-20
Leis,388,0000-00-00,1828-07-01
Teologia,353,0000-00-00,1884-05-26
Medicina,185,0000-00-00,1883-10-15
Filosofia,28,1770-10-01,1913-07-11
Matemática,24,1605-10-07,1913-07-11
Direito,22,1836-10-14,1898-12-18


### Faculdade corrigida

Casos em que o campo da faculdade estava preenchido mas era inconsistente com o resto da informação.

A maior parte dos casos dizem respeito a duplos percursos em Leis e Cânones e mudanças, depois de 1772, 
de faculdades de Matemática e Filosofia, que eram pre-requisitos, para as faculdades de posterior
matrículas (Leis, Cânones, Medicina, Teologia).

---

### "Faculdade" corrected

Cases where the algoritm changed the value of the field, because it was inconsistent with
the rest of information on the record. Mostly double degres in Canon and Civil law, and
changes from pre-requisite faculty to final graduation faculty.


In [6]:
fac_corr = notas_proc[notas_proc['nota-processamento']=='Aviso: faculdade corrigida'].copy()
fac_corr.dropna(how='all', axis=1, inplace=True)

In [7]:
import pandas as pd

df = fac_corr
col = 'nota-processamento.obs' # subotal by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                        'uc-entrada':'min',
                                        'uc-saida':'max'})
total = df_totals['id'].sum()
print("List of corrected 'faculdade'")
print("Total:",total)
df_totals['perc'] = df_totals['id']/total
pd.set_option('display.max_rows',1000)
df_totals.sort_values('id',ascending=False, inplace=True)
df_totals.head(10)

List of corrected 'faculdade'
Total: 6971


,id,uc-entrada,uc-saida,perc
nota-processamento.obs,,,,
"Faculdade corrigida de ""Cânones"" para ""Cânones, Leis"". .",1241,1536-10-00,1821-11-12,0.178023
"Faculdade corrigida de ""Leis"" para ""Cânones, Leis"". .",1240,1537-00-00,1829-06-13,0.177880
"Faculdade corrigida de ""Matemática"" para ""Medicina"".",1132,1772-11-28,1909-11-11,0.162387
"Faculdade corrigida de ""Direito"" para ""Leis"".",1009,1687-01-05,1827-06-16,0.144743
"Faculdade corrigida de ""Filosofia"" para ""Medicina"".",453,1540-10-25,1909-11-09,0.064984
"Faculdade corrigida de ""Teologia"" para ""Direito, Teologia"". .",231,1840-10-09,1915-10-22,0.033137
"Faculdade corrigida de ""Medicina"" para ""Cânones, Medicina"". .",202,1569-11-20,1771-07-30,0.028977
"Faculdade corrigida de ""Direito"" para ""Cânones"".",185,1566-10-07,1835-07-20,0.026539
"Faculdade corrigida de ""Teologia"" para ""Cânones, Teologia"". .",164,1540-10-03,1822-10-31,0.023526


In [8]:
fac_corr_nodup = fac_corr.drop_duplicates(subset=['id', 'faculdade-original'], keep='first')

In [9]:
show_only = 5
for  change,row in list(df_totals.iterrows())[:10]:
    print(f"{change}: {row.id}")
    for _ndx, row in fac_corr_nodup[fac_corr_nodup['nota-processamento.obs']==change][['id','name','url','uc-entrada']].head(show_only).iterrows():
        url = row.url.strip('"')
        print(f"     {row.id:6} {row['name']} {row['uc-entrada']} {url}")


Faculdade corrigida de "Cânones" para "Cânones, Leis". . : 1241
     140543 António de Abreu 1624-10-26 https://pesquisa.auc.uc.pt/details?id=140543
     140695 Duarte de Abreu 1566-01-20 https://pesquisa.auc.uc.pt/details?id=140695
     140772 Francisco Nunes de Abreu 1677-10-01 https://pesquisa.auc.uc.pt/details?id=140772
     140791 Gaspar de Abreu 1605-10-12 https://pesquisa.auc.uc.pt/details?id=140791
     140847 Valério de Abreu 1624-10-30 https://pesquisa.auc.uc.pt/details?id=140847
Faculdade corrigida de "Leis" para "Cânones, Leis". . : 1240
     140425 João de Abranches 1729-10-29 https://pesquisa.auc.uc.pt/details?id=140425
     140529 Ambrósio de Abreu 1653-10-15 https://pesquisa.auc.uc.pt/details?id=140529
     140617 António de Sousa de Abreu 1703-10-01 https://pesquisa.auc.uc.pt/details?id=140617
     140780 Francisco da Silva e Abreu 1739-10-01 https://pesquisa.auc.uc.pt/details?id=140780
     140790 Gaspar de Abreu 1610-10-05 https://pesquisa.auc.uc.pt/details?id=140790

### "Valor não corresponde a Faculdade nesta data"

Esta lista é produzida pela comparação do valor do campo de faculdade, ou outra referência a faculdade no registo, e a lista das faculdades existentes à data do registo.

Para além dos erros de ortografia e gralhas (atenção a "Cãnones" com til), também contém:

* Uma grande quantidade (>300) de campos de Faculdade com "?". Verificar se é porque falta informação na ficha, ou se existe outra explicação
* valores confundidos com Faculdade por particularidades da pontuação do registo. Nesse caso a correção a fazer pode ser na pontuação. 
   Ver por exemplo: 152975 a falta de ":" a seguir a matrícula provoca um erro no processamento da Faculdade
* valor que correspondem a faculdades que existiram, mas não naquelas datas (por exemplo Filosofia antes de 1772 ou Cânones depois de 1834).
* Topónimos, que serão erros de digitação que se poderão corrigir vendo a ficha original.

---

### Value does not correspond to a "Faculdade" at the time

This list is produced by comparing the value of the faculty field, or other reference to faculties in the record, and the list of faculties existing at the date of the record.

Apart from spelling errors and typos (watch out for tilted "Cãnones"), it also contains:

* A large amount (>300) of Faculty fields with "?". Check if this is because the information is missing in the original card, or if there is another explanation.
* Values confused with Faculty due to particularities of the register punctuation. In this case, the correction to be made may be in the punctuation. 
   See for example: 152975 the lack of ":" after the enrolment causes an error in the processing of the Faculty
* value corresponding to faculties that existed but not on those dates (e.g. Philosophy before 1772 or "Cânones" after 1834).
* Toponyms, which are typing errors that can be corrected by looking at the original form.

In [10]:
fac_error = notas_proc[notas_proc['nota-processamento'].str.contains('Erro:')].copy()
fac_error.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1065 entries, 140542 to 195743
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id_1                      1065 non-null   object
 1   name                      1065 non-null   object
 2   nota-processamento.type   1065 non-null   object
 3   nota-processamento        1065 non-null   object
 4   nota-processamento.date   1065 non-null   object
 5   nota-processamento.line   1065 non-null   int64 
 6   nota-processamento.level  1065 non-null   int64 
 7   nota-processamento.obs    1065 non-null   object
 8   faculdade                 390 non-null    object
 9   faculdade.date            390 non-null    object
 10  faculdade.obs             390 non-null    object
 11  faculdade-original        79 non-null     object
 12  faculdade-original.date   79 non-null     object
 13  faculdade-original.obs    79 non-null     object
 14  uc-entrada            

In [11]:
# currently the algorithm removed question marks, but they were there
# we have to put them back until this is fixed
fac_org_none = fac_error['faculdade-original'].isnull()
fac_error.loc[fac_org_none,'faculdade-original'] = '?'


In [12]:
df = fac_error
col = 'faculdade-original' # subotal by this column
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc-entrada':'min',
                                                  'uc-saida':'max'})

df_totals.sort_values('id',ascending= False).head(30)

,id,uc-entrada,uc-saida
faculdade-original,,,
?,986,0000-00-00,1912-08-02
Cânones,14,1843-01-16,1913-08-13
.,13,1604-10-17,1732-11-10
Leis,12,1836-06-04,1909-08-03
Intituta,7,1581-03-12,1770-10-01
Instituto,7,1694-12-15,1818-10-24
Dialética,4,1540-03-09,1542-03-06
Institua,4,1642-10-28,1733-10-01
Estado Eclesiástico,3,1846-11-11,1865-10-11


In [13]:
import pandas as pd
import csv

pd.set_option('display.max_rows',600)
exportar = fac_error[['faculdade','name','faculdade.date']].sort_values(['faculdade','name','faculdade.date'])
exportar.to_csv('../inferences/validation/faculdade-unkown.csv',sep=',')
exportar.head(10)

,faculdade,name,faculdade.date
id,,,
146645,.,António Antunes,1656-10-20
140542,.,António de Abreu,1661-10-26
165702,.,Manuel Ribeiro,1604-10-17
165556,.,Manuel Ribeiro,1612-03-23
165558,.,Manuel Ribeiro,1617-04-09
165781,.,Manuel Ribeiro,1657-10-15
165804,.,Manuel Ribeiro,1659-12-12
165730,.,Manuel Ribeiro,1683-03-24
165828,.,Manuel Ribeiro,1689-11-24


### "Faculdades" com menos de 100 ocorrências. 

---

Parece haver um número elevado de casos que iniciam o curso pelos anos 1820 em Leis e depois acabam em Direito, nos anos 1830.

* https://pesquisa.auc.uc.pt/details?id=149500
* https://pesquisa.auc.uc.pt/details?id=128134
* https://pesquisa.auc.uc.pt/details?id=141618
* https://pesquisa.auc.uc.pt/details?id=145263

Fazer uma pesquisa por todos os entrados em 1820/9 e saídos em 1830/9

### "Faculdades" with less than 100 stduents

In [14]:

from timelink.pandas import attribute_values


facs = attribute_values('faculdade',
                        dates_between=('1500-00-00','1990-00-00'),
                        db=db)
facs[facs['count']<100]

,count,date_min,date_max
value,,,
.,16,1604-10-17,1738-12-01
Instituto,7,1694-12-15,1818-10-24
Intituta,7,1581-03-12,1770-10-01
Dialética,5,1540-03-09,1540-10-25
Institua,4,1642-10-28,1733-10-01
Cãnones,3,1572-10-01,1626-10-06
Estado Eclesiástico,3,1846-11-11,1865-10-11
Canônes,2,1576-10-01,1609-11-23
Farmácia,2,1903-01-14,1904-10-12


Todo:

This could be generalized as:
    to_markdown(list_of_ids: list, generate_index=True, index_title="My Index", filename_pattern="{id} {name})

In [15]:
import os
from timelink.pandas import entities_with_attribute
from ucalumni.aluno import get_and_process_aluno


path = '../database/quality_assurance/faculdade/less_than_100'
index_file = f"00 Lista.md"
index_file_clean = index_file.replace(" ", "%20")
record_template = """
Voltar ao [índice]({index_file})
{aluno_canonic}
### Informação de processamento
{aluno_ficha}
"""
# create directory if it does not exist
if not os.path.exists(path):
    os.makedirs(path)
# create file index.md in the directory
with open(f"{path}/{index_file}", 'w') as file:
    file.write("# Faculdades com menos de 100 entradas na base de dados\n\n")

facs_errors = facs[facs['count']<100]
show_only = 100
for f,row in facs_errors.sort_values('count',ascending=False).iterrows():
        print()
        print(f)
        with open(f"{path}/{index_file}", 'a') as file:
                file.write(f"## {f}\n\n")
        students = entities_with_attribute(
                entity_type='person',
                the_type='faculdade',
                the_value=f,
                show_elements=['id','name'],
                more_attributes=['nota-processamento','uc-entrada'],
                db=db)
        last_linked_aluno = ''
        for id, aluno in students.sort_values(['name','uc-entrada']).head(show_only).iterrows():
                aviso = aluno["nota-processamento"]
                if aviso is not None:
                        aviso = aviso + " " + aluno['nota-processamento.obs']
                else:
                        aviso = ""
                id_link = f'<a href="https://pesquisa.auc.uc.pt/details?id={id}">{id}</a>'
                # display(HTML(f'<code>{row.the_value:24} [{id_link:5}] {aluno.the_date} {aluno.name:44}</code>'))
                aluno_linha = f"{aluno['faculdade']:12} [{id:5}] {aluno['faculdade.date']} {aluno['name']:44} {aviso} https://pesquisa.auc.uc.pt/details?id={id} {aluno['faculdade.obs']}"
                # print(aluno_linha)
                aluno_ficha = get_and_process_aluno(id, db=db)
                filename = f"{id} {aluno_ficha.nome}.md"
                with open(f"{path}/{filename}", 'w') as file:
                        aluno_canonic = aluno_ficha.canonic()
                        aluno_record = str(aluno_ficha)
                        file.write(record_template.format(index_file=index_file_clean,
                                                          aluno_canonic=aluno_canonic,
                                                          aluno_ficha=aluno_ficha
                                                          )
                        )

                with open(f"{path}/{index_file}", 'a') as file:
                        # escape filename for markdown and spaces
                        filename_clean = filename.replace(" ", "%20")
                        if last_linked_aluno != id:
                                file.write(f"* [{id} {aluno_ficha.nome}]({filename_clean})\n     * {aviso}\n")
                                last_linked_aluno = id
                        else:
                                file.write(f"     * {aviso}\n")


.

Intituta

Instituto

Dialética

Institua

Cãnones

Estado Eclesiástico

Medicna

Teórico

Teorico

Farmácia

Canônes

Graduação

Teolggia

Matemátiica

Matemátca

Matematíca

Instiuta

Instituto Eclesiástico

Insituta

Granática

Coimbra

Fiosofia

Filosogia

Filosfia

Estado Elcesiástico

Eclesiástico

Codigo

Cantanhede

1767-10-01

Teóricos
